## Finding frequent itemsets with the Apriori algorithm

In [1]:
from numpy import *

**loadDataSet()** creates a simple dataset

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

C1 is a candidate itemset of
size one. In the Apriori algorithm, we create C1, and then we’ll scan the dataset to see
if these one itemsets meet our minimum support requirements. The itemsets that do meet our minimum requirements become L1. L1 then gets combined to become C2
and C2 will get filtered to become L2.

In [4]:
def creatC1(dataSet): 
    C1 = []
    for transaction in dataSet:
        for item in transaction: 
            if not[item] in C1: 
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)

**scanD()** takes three arguments:
a dataset, Ck, a list of candidate sets, and minSupport, which is the minimum
support you’re interested in. This is the function you’ll use to generate L1 from C1.
Additionally, this function returns a dictionary with support values for use later.

In [5]:
def scanD(D, Ck, minSupport): 
    
    ssCnt = {}
    
    Dlist = list(D)
    Cklist = list(Ck)
    
    for tid in Dlist:
        for can in Cklist: 
            if can.issubset(tid): 
                if can not in ssCnt: 
                    ssCnt[can] = 1
                else: 
                    ssCnt[can] += 1
    numItems = float(len(Dlist))
    
    retList = []
    supportData = {}
    
    for key in ssCnt: 
        support = ssCnt[key]/numItems 
        if support >= minSupport: 
            retList.insert(0, key) 
        supportData[key] = support
    
    return retList, supportData

In [13]:
D = map(set, dataSet)
C1 = creatC1(dataSet)

Four items make up our L1 list, that is, the list of one-item sets that occur in at
least 50% of all transactions. Item 4 didn’t make the minimum support level, so it’s
not a part of L1. 

In [14]:
L1, suppData0 = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

The function **aprioriGen()** will take a list of frequent itemsets, Lk, and the size of
the itemsets, k, to produce Ck.


Everything gets wrapped up in the **apriori() function**. You give this a dataset and
a support number, and it will generate a list of candidate itemsets.

In [8]:
def aprioriGen(Lk, k): 
    retList = []
    lenLk = len(Lk)
    
    for i in range(lenLk): 
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2].sort()
            L2 = list(Lk[j])[:k-2].sort()
            if L1 == L2: 
                retList.append(Lk[i] | Lk[j])
    return map(frozenset, retList)

In [9]:
def apriori(dataSet, minSupport = 0.5): 
    C1 = creatC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    
    while(len(L[k-2]) > 0):
        D = map(set, dataSet)
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    
    return L, supportData

L contains some lists of frequent itemsets that met a minimum support of 0.5(minsupport = 0.5 by default).

In [10]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

We can try algorithm by support = 0.7

In [12]:
L, suppData = apriori(dataSet, minSupport = 0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

## Mining association rules from frequent item sets

The **generateRules()** function takes three inputs: a list of frequent itemsets, a dictionary
of support data for those itemsets, and a minimum confidence threshold. It’s
going to generate a list of rules with confidence values that we can sort through later.
These rules are stored in bigRuleList.

In [13]:
def generateRules(L, supportData, minConf=0.7): 
    
    bigRuleList = []
    
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

We’re interested in calculating the confidence of a rule and then finding out
which rules meet the minimum confidence. All of this is done in **calcConf()**,

In [14]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    
    prunedH = []
    
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

To generate more association rules from our initial itemset, you use the
**rulesFromConseq() function.

In [15]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    
    m = len(H[0])
    #H中的频繁集大小
    
    '''
    接下来查看该频繁项集是否大到可以移除大小为m的子集。如果可以的话，则将其移除。
    '''
    
    if (len(freqSet) > (m + 1)): 
        
        Hmp1 = aprioriGen(H, m+1)
        #可以使用aprioriGen()来生成H中元素的无重复组合。
        
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        #可以利用calcConf()来测试它们的可信度以确定规则是否满足要求
        
        if (len(Hmp1) > 1): 
        #如果不止一条规则满足要求，递归调用来判断是否可以进一步组合这些规则。
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [16]:
L, suppData = apriori(dataSet, minSupport = 0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [17]:
rules = generateRules(L, suppData, minConf = 0.7)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 2.0
frozenset({3}) --> frozenset({2, 5}) conf: 2.0
frozenset({2}) --> frozenset({3, 5}) conf: 2.0
